# Yeast cells detection sample notebook

In [ ]:
import sys
sys.path.append('../')
#sys.path.append('%HOMEPATH%/detectron2') #insert local path to detecron2
import numpy as np

import yeastcells.clustering as clustering
import yeastcells.features as features
import yeastcells.data as data
import yeastcells.model as model
import yeastcells.output as output
import yeastcells.visualize as visualize

%matplotlib notebook

In [ ]:
cd ~

### Load data

In [ ]:
path = '../../slow-data/yeastcells/data/DATA_CIT/Test_Images'
filenames = data.load_data(path, ff = '.tif')
print(filenames)

In [ ]:
fn = filenames[0]
image = data.read_image(fn, single_im = False, shape=2)
np.shape(image) #(frames, length, width, channels)

In [ ]:
image = image[39:]
len(image)

### Load and run model on data for segmentation

In [ ]:
#set path to model_final.pth
model_path = '/var/tensorflow-logs/p253591/yeast-cell-detection-run-4'

#load model
predictor = model.load_model(model_path, seg_thresh=0.94, device='cuda:0') #set GPU if available otherwise use 'cpu'

In [ ]:
output = [
  {'instances': predictor(frame)['instances'].to('cpu')}
  for frame in image
]

### Run clustering algorithm for tracking

In [ ]:
labels, coordinates = clustering.cluster_cells(output, dmax=5, min_samples=5, eps=0.6, progress=False) #for time-series
#labels, coordinates = get_instance_numbers(output) #for single image

In [ ]:
#plot paths
ax = visualize.plot_paths(
  labels,
  coordinates[:, [2,1,0]],
  style={'linewidth': 1},
  xlim=(0, 512),
  ylim=(0, 512),
  title='',
)

### Extract features and visualize tracking and segmentation

In [ ]:
s, t = features.get_seg_track(labels, output, frame = None) # total

In [ ]:
s, t = features.get_seg_track(labels, output, frame = 0, start=40) # by frame

In [ ]:
features.track_len(labels, label=0) #select a label to extract the length of its track

In [ ]:
contours = features.extract_contours(output) 

In [ ]:
scene = visualize.create_scene(image, output, labels, contours, thickness=1, color=[255,0,255], framenum=True, labelnum=True)
movie = visualize.show_animation(scene, title='', delay=100) 

In [ ]:
scene = visualize.create_scene(image, output, labels, contours, thickness=1, color=None, framenum=True, labelnum=True)
cell_scene, label = visualize.select_cell(scene, coordinates, labels, w=60, l=0) # choose image window (w) and label (l)
movie = visualize.show_animation(cell_scene, title='', delay=100)

In [ ]:
polygons_clust =  features.polygons_per_cluster(labels, contours, output)

In [ ]:
ax = visualize.plot_area_profiles(polygons_clust, ti=3, label_list=[0], ax=None, title=None)

In [ ]:
_, _, pred_df = data.get_pred(output, labels, coordinates, ti=3, start=40)
polygons_inst =  features.polygons_per_instance(labels, contours, output)
pred_df = features.add_area_df(polygons_inst, labels, pred_df, start=40)

In [ ]:
masks = features.get_masks(output)

In [ ]:
visualize.plot_polygon_mask(
    masks, labels, output, image, polygons_clust, 
    label=0, frame=0, ax=None, title=None
)

In [ ]:
agr = features.get_average_growth_rate(polygons_clust, labels, output)
area_std = features.get_area_std(polygons_clust, labels, pred_df)
position_std = features.get_position_std(polygons_clust, labels, pred_df)

### Output

In [ ]:
pred_df.head(20)